Hellan-Herrmann-Johnson Method
====

In [ ]:
from ngsolve import *
from netgen.geom2d import unit_square
import netgen.gui
%gui tk

Define product finite element space for $H(\text{div div}) \times H^1$

In [ ]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.1))
Draw (mesh)

order = 0
Sigma = HDivDiv(mesh, order=order)
W = H1(mesh, order=order+1, dirichlet="top|bottom|left|right")
X = FESpace( [Sigma,W] )

print ("ndof-Sigma:", Sigma.ndof, ", ndof-W:", W.ndof, ", ndof-X:", X.ndof)

$$A(\sigma,u) = -\frac{1}{2}a(\sigma,\sigma) + b(\sigma,u)$$
with
\begin{align*}
a(\sigma,\tau) & =  \int_\Omega \sigma: \tau\,dx \\
b(\sigma, v) & =  \sum_T \int_T \text{div} \sigma \cdot \nabla v\,dx - \int_{\partial T} \sigma_n\cdot \nabla_t v\,ds
\end{align*}

In [ ]:
sigma, u = X.TrialFunction()

n = specialcf.normal(2)
def tang(u): return u-(u*n)*n

a = BilinearForm(X, symmetric=True)
a += SymbolicEnergy (-0.5*InnerProduct(sigma, sigma) + div(sigma)*grad(u)) 
a += SymbolicEnergy (-(sigma*n) * tang(grad(u)), element_boundary=True)
a += SymbolicEnergy (-500 * u)
                    
gfu = GridFunction(X)
r = gfu.vec.CreateVector()

a.Apply(gfu.vec, r)
a.AssembleLinearization(gfu.vec)
gfu.vec.data -= a.mat.Inverse(X.FreeDofs()) * r

Draw (gfu.components[0], mesh, "sigma")
Draw (gfu.components[1], mesh, "disp")